In [ ]:
# https://huggingface.co/am-azadi/EXIST2024_Task1_xlmRoberta_large_3_16_merged_unshuffled

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from statistics import mode

# Load the tokenizer and model
model_name = "am-azadi/EXIST2024_Task1_xlmRoberta_large_3_16_merged_unshuffled"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the dataset
with open("EXIST2024_training.json", "r") as file:
    data = json.load(file)

# Function to preprocess and predict
def predict_sexism(tweet):
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, max_length=128)
    outputs = model(**inputs)
    logits = outputs.logits
    return torch.argmax(logits, dim=-1).item()
    # 1 is "yes", 0 is "no"

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
# Evaluation on the dataset
TP = 0
TN = 0
FP = 0
FN = 0

# keep only the 10 first tweets to test the implementation
subset_data = dict(list(data.items())[:30])  # Create a subset of the data
# subset_data = data  # Use the entire dataset

count = 0

for item_id, item_data in subset_data.items():
    tweet = item_data["tweet"]
    ground_truth = 1 if item_data["labels_task1"].count("YES") >=4 else 0
    model_output = predict_sexism(tweet)

    if model_output == ground_truth:
      if ground_truth == 1:
        TP += 1
      else:
        TN += 1
    else:
      if ground_truth == 0:
        FP += 1
      else:
        FN += 1

    # Debug print
    # print(f"Tweet: {tweet}")
    # print(f"Ground Truth: {ground_truth} | Labels: {item_data['labels_task1']}")
    # print(f"Model Output: {model_output}")
    # print("-" * 50)
    if count==20 :
      print(".", end="")
      count = 0
    count += 1

total_predictions = TP + TN + FP + FN
accuracy = (TP + TN) / total_predictions if total_predictions > 0 else 0
f1_score = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

print()
print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
print(f"Accuracy: {accuracy:.2%}")
print(f"F1 Score: {f1_score:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")

....
TP: 26, TN: 58, FP: 16, FN: 0
Accuracy: 84.00%
F1 Score: 76.47%
Precision: 61.90%
Recall: 100.00%
